# Significant Collocations

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn import set_config

from nltk.collocations import QuadgramCollocationFinder
from nltk.metrics.association import QuadgramAssocMeasures

tqdm.pandas()
set_config(display="diagram")

In [2]:
df = pd.read_csv('../input/sentiment-analysis-for-financial-news/all-data.csv', delimiter=',', encoding='latin-1', header=None).fillna('')
df = df.rename(columns=lambda x: ['sentiment', 'text'][x])
print(df.shape)
df.head()

(4846, 2)


,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [3]:
class SignificantCollocations(BaseEstimator, TransformerMixin):
    def __init__(self, ngram_class=QuadgramCollocationFinder, metric=QuadgramAssocMeasures.pmi):
        self.ngram_class = ngram_class
        self.metric = metric
    
    def fit(self, X, y=None):
        ngrams = self.ngram_class.from_documents(X)
        self.scored_ = dict(ngrams.score_ngrams(self.metric))
        return self
    
    def transform(self, X, y=None):
        for x in tqdm(X):
            ngrams = self.ngram_class.from_words(x)
            yield {
                ngram: self.scored_.get(ngram, 0.0) for ngram in ngrams.nbest(QuadgramAssocMeasures.raw_freq, 50)
            }
    

In [4]:
get_model = lambda: Pipeline([
    ('vectorizer', FeatureUnion([
        ('collocations', Pipeline([('extractor', SignificantCollocations()), ('vectorizer', DictVectorizer())])),
        ('vanilla', TfidfVectorizer(sublinear_tf=True))
    ])),
    ('estimator', LogisticRegression(max_iter=100_000, random_state=19, class_weight="balanced"))
])
get_model()

Pipeline(steps=[('vectorizer',
                 FeatureUnion(transformer_list=[('collocations',
                                                 Pipeline(steps=[('extractor',
                                                                  SignificantCollocations()),
                                                                 ('vectorizer',
                                                                  DictVectorizer())])),
                                                ('vanilla',
                                                 TfidfVectorizer(sublinear_tf=True))])),
                ('estimator',
                 LogisticRegression(class_weight='balanced', max_iter=100000,
                                    random_state=19))])

In [5]:
x = df['text']
y = df['sentiment']

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)
scores = []
for train_index, valid_index in tqdm(skf.split(x, y), total=10):
    x_train, x_valid = x.iloc[train_index], x.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    model = get_model().fit(x_train, y_train)
    scores.append(model.score(x_valid, y_valid))
print(np.mean(scores))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4361 [00:00<?, ?it/s]

  0%|          | 0/485 [00:00<?, ?it/s]

  0%|          | 0/4361 [00:00<?, ?it/s]

  0%|          | 0/485 [00:00<?, ?it/s]

...

  0%|          | 0/4362 [00:00<?, ?it/s]

  0%|          | 0/484 [00:00<?, ?it/s]

  0%|          | 0/4362 [00:00<?, ?it/s]

  0%|          | 0/484 [00:00<?, ?it/s]

0.7071836925960637
